In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


# Step 1: Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model.eval()

# Step 2: Load the images
input_directory = "./output_images_folder_50000"
output_original_directory = "./original_incorrect_predictions_folder(FGSM)"
output_adversarial_directory = "./adversarial_images_folder(FGSM)"

input_images = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith(".JPEG")]

# Step 3: Define the FGSM attack function
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# Step 4: Generate adversarial examples
epsilon = 0.01  # Adjust this value as needed
successful_adversarial_examples = []
corres_org_img=[]

for input_image in input_images:
    # Load the original image
    original_image = Image.open(input_image).convert("RGB")
    
    # Convert the PIL image to a PyTorch tensor and apply the necessary preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    image_tensor = preprocess(original_image).unsqueeze(0)
    image_tensor.requires_grad = True

    # Forward pass to get the predictions
    output = model(image_tensor)
    original_label = torch.argmax(output, 1).item()

    # Calculate the loss
    loss = nn.CrossEntropyLoss()(output, torch.tensor([original_label]))

    # Zero all existing gradients
    model.zero_grad()

    # Backpropagate the loss to compute gradients
    loss.backward()

    # Collect the gradients of the input image
    data_grad = image_tensor.grad.data

    # Generate the perturbed image using the FGSM attack
    perturbed_image = fgsm_attack(image_tensor, epsilon, data_grad)

    # Forward pass with the perturbed image
    perturbed_output = model(perturbed_image)

    # Check if the perturbed image's predicted label differs from the original label
    
    perturbed_label = torch.argmax(perturbed_output, 1).item()
    if perturbed_label != original_label:
        # Save the perturbed image as a successful adversarial example
        successful_adversarial_examples.append(perturbed_image)
        corres_org_img.append(input_image)
        # Save the original image to output_original_directory
        os.makedirs(output_original_directory, exist_ok=True)
        original_image_name = os.path.basename(input_image)
        output_original_image_path = os.path.join(output_original_directory, original_image_name)
        original_image.save(output_original_image_path)
        
        # Save the perturbed image to output_adversarial_directory
        os.makedirs(output_adversarial_directory, exist_ok=True)
        perturbed_image_name = f"perturbed_{original_image_name}"
        output_perturbed_image_path = os.path.join(output_adversarial_directory, perturbed_image_name)
        perturbed_image_pil = transforms.ToPILImage()(perturbed_image.squeeze(0))
        perturbed_image_pil.save(output_perturbed_image_path)

# Print the length of successful adversarial examples
print("Number of successful adversarial examples:", len(successful_adversarial_examples))
print(len(corres_org_img))

# # Show the original and adversarial images for all successful examples
# for idx, perturbed_image in enumerate(successful_adversarial_examples):
#     # Load the original image
#     original_image_path = corres_org_img[idx]
#     try:
#         original_image = Image.open(original_image_path).convert("RGB")
#     except Exception as e:
#         print(f"Error loading original image {original_image_path}: {e}")
#         continue

#     # Convert the perturbed image to a numpy array
#     perturbed_image_np = perturbed_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0)

#     # Get the original label
#     with torch.no_grad():
#         original_image_tensor = preprocess(original_image).unsqueeze(0)
#         original_output = model(original_image_tensor)
#         original_label_idx = torch.argmax(original_output, 1).item()

#     # Get the adversarial label
#     with torch.no_grad():
#         perturbed_output = model(perturbed_image)
#         perturbed_label_idx = torch.argmax(perturbed_output, 1).item()

#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))

#     # Plot the original image
#     axes[0].imshow(original_image)
#     axes[0].set_title(f"Original Image {idx+1}, Label: {original_label_idx}")
#     axes[0].axis('off')

#     # Plot the adversarial image
#     axes[1].imshow(perturbed_image_np)
#     axes[1].set_title(f"Adversarial Image {idx+1}, Label: {perturbed_label_idx}")
#     axes[1].axis('off')

#     plt.show()

C:\Users\DandaC8719\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DandaC8719\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import json

# Step 1: Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model.eval()

# Load ImageNet class labels
with open("C:/Users/DandaC8719/Documents/capstone_overall_code/imagenet_class_index.json", "r") as f:
    imagenet_classes = json.load(f)

# Convert class labels to dictionary format
imagenet_classes = {int(k): v for k, v in imagenet_classes.items()}

# Step 3: Define the FGSM attack function
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image
    
# Step 4: Generate adversarial examples
epsilon = 0.01  # Adjust this value as needed

def get_labels(image_path):
    from PIL import Image  # Import Image module here
    
    # Load the original image
    original_image = Image.open(image_path).convert("RGB")
    
    # Convert the PIL image to a PyTorch tensor and apply the necessary preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    image_tensor = preprocess(original_image).unsqueeze(0)
    image_tensor.requires_grad = True

    # Forward pass to get the predictions
    output = model(image_tensor)
    original_label_id = torch.argmax(output, 1).item()

    # Calculate the softmax probabilities for the predictions
    probabilities = nn.functional.softmax(output, dim=1)[0]

    # Get class names for original label
    original_label_name = imagenet_classes[original_label_id]

    # Get the confidence percentage for the original prediction
    original_confidence = probabilities[original_label_id].item() * 100

    # Calculate the loss
    loss = nn.CrossEntropyLoss()(output, torch.tensor([original_label_id]))

    # Zero all existing gradients
    model.zero_grad()

    # Backpropagate the loss to compute gradients
    loss.backward()

    # Collect the gradients of the input image
    data_grad = image_tensor.grad.data

    # Generate the perturbed image using the FGSM attack
    perturbed_image = fgsm_attack(image_tensor, epsilon, data_grad)

    # Forward pass with the perturbed image
    perturbed_output = model(perturbed_image)

    # Get the adversarial label
    perturbed_label_id = torch.argmax(perturbed_output, 1).item()

    # Get class names for adversarial label
    perturbed_label_name = imagenet_classes[perturbed_label_id]
    
    # Calculate the softmax probabilities for the adversarial predictions
    perturbed_probabilities = nn.functional.softmax(perturbed_output, dim=1)[0]

    # Get the confidence percentage for the adversarial prediction
    perturbed_confidence = perturbed_probabilities[perturbed_label_id].item() * 100
    
    return (original_label_id, original_label_name, original_confidence), (perturbed_label_id, perturbed_label_name, perturbed_confidence)

# Example usage:
image_path = "./output_images_folder_50000/ILSVRC2012_val_00040021.JPEG"  
(original_label_id, original_label_name, original_confidence), (adversarial_label_id, adversarial_label_name, adversarial_confidence) = get_labels(image_path)
print("Original Label ID:", original_label_id)
print("Original Label Name:", original_label_name)
print("Original Confidence:", original_confidence)
print("Adversarial Label ID:", adversarial_label_id)
print("Adversarial Label Name:", adversarial_label_name)
print("Adversarial Confidence:", adversarial_confidence)

Original Label ID: 985
Original Label Name: ['n11939491', 'daisy']
Original Confidence: 97.21648097038269
Adversarial Label ID: 301
Adversarial Label Name: ['n02165456', 'ladybug']
Adversarial Confidence: 13.377849757671356
